# Mochila Quadrática $0$-$1$ (QKP)

## Dados

Considere dos seguintes dados:

-  $N = \{1, \ldots, n\}$ um conjunto dos objetos.

-  $P = \{p_{ij} \in \mathbb{R}_{+} : \ i \leq j, \ i, j \in N \}$ o conjunto dos benefícios aos itens de $N$ e a relação entre eles.

-  $W = \{ w_i \in \mathbb{N} : i \in N \}$ o conjunto dos pesos dos itens de $N$.

-  $c \in \mathbb{N}$ capacidade da mochila, onde $\displaystyle\max_{i \in N} w_i \leq c < \sum_{i  \in N} w_i$.

## QKP 0-1 linear

Utilizando os dados acima para definir o problema da mochila quadrática 0-1.

\begin{align}
max \; & \sum_{i \in N} q_{i} x_i + \sum_{i = 1}^{n-1} \sum_{j =i+1}^{n}
p_{ij} y_{i,j} \\
& \sum_{i \in N} w_{i} x_{i} \leq c, \\
&  y_{i,j} \leq x_{i}, \\
&  y_{i,j} \leq x_{j}, \\
&  x_{i} + x_{j} \leq y_{i,j} + 1, \\
& x_{i} \in \left\{0, 1 \right\}, \ \ i  \in N.
\end{align}

onde $q_{i} = p_{ii}, \ i \in N$.

**Callback:** função chamada periodicamente no meio da otimização, permitindo modificar e/ou monitorar o estado da otimização.

**Argumentos da função de callback:** 

- modelo

- **where**, indica onde a função de callback será chamada.  
Para utilizá-la é necessário adicionar GRB.Callback mais o código referente ao where desejado. 
É importante definir o que você quer fazer (what), para o termo what há um número muito grande de informações que podem ser obtidas que estão diretamente ligadas com o where utilizado. 
Para utilizá-la é necessário adicionar a função Model.cbGet( ) e como argumento dessa função passar o GRB.Callback mais o código referente a informação desejada.

Callbacks are used to report on the progress of the optimization or to modify the behavior of the Gurobi solver. 

To use a callback, the user writes a routine that implements the desired behavior. 

The routine is passed to the Gurobi Optimizer when optimization begins, and the routine is called regularly during the optimization process. 

One argument of the user routine is a **where value**, which indicates from where in the optimization process the callback is invoked. 

The user callback routine can call the optimization library to query certain values. 

## Modelando o problema e resolvendo com gurobipy

In [3]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [4]:
def read_instance(dim,perc,t):

    instance = f"instances/{dim}/{dim}_{perc}_{t}.txt"

    with open(instance, 'r') as file: linhas = file.readlines()

    # remove linha vazia inicial e elimina os "\n" de cada linha
    linhas = [a.strip() for a in linhas] 

    # ler o tamanho da instancia
    n = int(linhas[0]) 

    # ler a diagonal da matriz
    d = np.fromstring(linhas[1], dtype=int, sep = ' ') 

    # define a matriz
    p = np.zeros((n,n), dtype=int) 

    # preenche a diagonal
    for i in range(n): 
        p[i][i] = d[i]

    # preenche o resto da matriz
    for i in range(n-1): 
        linha = np.fromstring(linhas[i+2], dtype=int, sep = ' ')
        for j in range(n-(i+1)):
            p[i][j+i+1] = linha[j]
            p[j+i+1][i] = p[i][j+i+1]

    # ler a capacidade
    c = int(linhas[n+2]) 

    # ler os pesos
    w = np.fromstring(linhas[n+3], dtype=int, sep = ' ') 

    return n, p, w, c

In [12]:
def bc_callback(model, where):
  
  if where == GRB.Callback.MIPNODE:

    x_var,y_var = model.__data
    n = len(x_var)
    
    # MIP node callback
    status = model.cbGet(GRB.Callback.MIPNODE_STATUS)
    
    print('**** New node ****')    
    
    if status == GRB.OPTIMAL:
      print("status = ", status)
      x = model.cbGetNodeRel(model._vars)
      model.cbSetSolution(model.getVars(), x)
      
      #print("tam x:", len(x))
      #print(x)
      
      #print("n = ", n)
      k = n-1          
      for i in range(0,n):
        for j in range(i+1, n):
          k = k + 1 
          #print("i %d, j %d , k %d " %(i,j,k))
          if (x[i] + x[j] > 1 + x[k]):
            print("corte adicionado: x[%d] + x[%d] <= 1 + y[%d,%d]" %(i,j,i,j))
            model.cbCut(model._vars[i] + model._vars[j] <= 1 + model._vars[k])

  #if where == GRB.Callback.MIPSOL:
  #  print("Time(s):",model.cbGet(GRB.Callback.RUNTIME),
  #  "best solution:",model.cbGet(GRB.Callback.MIPSOL_OBJBST),
  #  "best bound:",model.cbGet(GRB.Callback.MIPSOL_OBJBND))


In [13]:
def qknapsack(dim,perc,t):

    # read instance
    n, p, w, c = read_instance(dim, perc, t)

    #cria o modelo
    model = gp.Model("qkp_linear") 

    x = []
    for j in range(0, n):
        x.append(model.addVar(vtype=GRB.BINARY, name="x_{}".format(j+1)))

    l = list(tuple())
    for i in range(0, n):
        for j in range(i+1, n):
            l.append((i,j))

    y = model.addVars(l, vtype=GRB.BINARY, name='y') #Adicionando Variáveis 
        
    obj = 0 
    for i in range(0, n):
        obj += p[i][i] * x[i]
        for j in range(i+1, n):
            obj += p[i][j] * y[i,j]

    model.setObjective(obj, GRB.MAXIMIZE)

    constr = 0
    for j in range(0, n):
        constr += (w[j] * x[j])
    model.addConstr(constr <= c)

    for i in range(0,n):
        for j in range(i+1, n):
            constr1 = y[i,j]
            model.addConstr(constr1 <= x[i])

    for i in range(0,n):
        for j in range(i+1, n):
            constr2 = y[i,j]
            model.addConstr(constr2 <= x[j])

    #for i in range(0,n):
    #    for j in range(i+1, n):
    #        constr3 = x[i] + x[j]
    #        model.addConstr(constr3 <= 1 + y[i,j])
            
    model.__data = x,y

    model.write(f"lp/qkp_bc_linear_{dim}_{perc}_{t}.lp")

    # configurando parametros
    #model.Params.IterationLimit = 1000 # define o número de iterações do simplex
    #model.Params.TimeLimit = 60 # define tempo limite
    model.Params.MIPGap = 1.e-6 # define valor do gap
    #model.Params.method = 1
    #model.Params.NodeMethod = -1 #  -1=automatic, 0=primal simplex, 1=dual simplex, and 2=barrier
    model.Params.Threads = 1
    model.Params.Presolve = 0
    model.Params.Cuts = 0
    model.Params.PreCrush = 1
    
    # Turn off display
    gp.setParam('OutputFlag', 0)
    # Turn off heuristics
    #gp.setParam('Heuristics', 0)

    # Open log file
    logfile = open('bc_callback.log', 'w')
 
    # Pass data into my callback function
    model._lastiter = -GRB.INFINITY
    model._lastnode = -GRB.INFINITY
    model._logfile = logfile
    model._vars = model.getVars()

    model.optimize(bc_callback)

    #model.optimize()
    
    status = 0
    if model.status == GRB.OPTIMAL:
        status = 1
 
    ub = model.objBound
    lb = model.objVal
    gap = model.MIPGap
    time = model.Runtime
    nodes = model.NodeCount

    model.dispose()

    #lower bound, upper bound, gap, time, nodes
    arquivo = open(f'result/qkp_linear_{dim}_{perc}.csv','a')
    arquivo.write(
        str(f"{dim}_{perc}_{t}")+';'
        +str(round(lb,1))+';'
        +str(round(ub,1))+';'
        +str(round(gap,2))+';'
        +str(round(time,2))+';'
        +str(round(nodes,1))+';'
        +str(round(status,1))+'\n'
    )
    arquivo.close()

In [14]:
# __main__ : “top-level code environment”

# What is the “top-level code environment”?
# __main__ is the name of the environment where top-level code is run. 
# “Top-level code” is the first user-specified Python module that starts running. 
# It’s “top-level” because it imports all other modules that the program needs. 
# Sometimes “top-level code” is called an entry point to the application.

if __name__ == "__main__":

    for dim in [50]:
        for perc in [75]:
            for t in range(1,2):
                instance = f"instances/{dim}/{dim}_{perc}_{t}.txt"
                qknapsack(dim,perc,t)

**** New node ****
status =  2
corte adicionado: x[0] + x[6] <= 1 + y[0,6]
corte adicionado: x[0] + x[9] <= 1 + y[0,9]
corte adicionado: x[0] + x[25] <= 1 + y[0,25]
corte adicionado: x[0] + x[26] <= 1 + y[0,26]
corte adicionado: x[0] + x[36] <= 1 + y[0,36]
corte adicionado: x[1] + x[6] <= 1 + y[1,6]
corte adicionado: x[1] + x[8] <= 1 + y[1,8]
corte adicionado: x[1] + x[11] <= 1 + y[1,11]
corte adicionado: x[1] + x[19] <= 1 + y[1,19]
corte adicionado: x[1] + x[33] <= 1 + y[1,33]
corte adicionado: x[1] + x[36] <= 1 + y[1,36]
corte adicionado: x[1] + x[43] <= 1 + y[1,43]
corte adicionado: x[2] + x[4] <= 1 + y[2,4]
corte adicionado: x[2] + x[14] <= 1 + y[2,14]
corte adicionado: x[2] + x[17] <= 1 + y[2,17]
corte adicionado: x[2] + x[19] <= 1 + y[2,19]
corte adicionado: x[2] + x[25] <= 1 + y[2,25]
corte adicionado: x[2] + x[28] <= 1 + y[2,28]
corte adicionado: x[2] + x[29] <= 1 + y[2,29]
corte adicionado: x[2] + x[42] <= 1 + y[2,42]
corte adicionado: x[2] + x[43] <= 1 + y[2,43]
corte adicion

## Tabelando os resultados 

In [5]:
import pandas as pd

dim = 50
perc = 75
inst_=f'result/qkp_linear_{dim}_{perc}.txt'

df = pd.DataFrame()
df = pd.read_csv(inst_,header=None,sep=';')

tab = pd.DataFrame()
tab = pd.concat([tab, df], ignore_index=True)
tab.columns = ['instance','bs','bb','gap','time','nodes','opt']

resume = pd.DataFrame({
    'instance':f"resume",
    'bs':tab["bs"].mean(),
    'bb':tab["bb"].mean(),
    'gap':tab['gap'].mean(),
    'time':tab['time'].mean(),
    'nodes':tab['nodes'].mean(),
    'opt':tab['opt'].sum(),
     },index=[f"qkp"]
)

tab = pd.concat([tab, resume], ignore_index=True)

tab["bb"] = tab["bb"].round(2)
tab["bs"] = tab["bs"].round(2)
tab["gap"] = tab["gap"].round(2)
tab["time"] = tab["time"].round(2)
tab["nodes"] = tab["nodes"].round(2)
tab["opt"] = tab["opt"].round().astype('Int64')

tab

,instance,bs,bb,gap,time,nodes,opt
0,50_75_1,14162.0,14162.0,0.0,0.77,27.0,1
1,50_75_2,12226.0,12226.0,0.0,0.76,64.0,1
2,50_75_3,23254.0,23254.0,0.0,0.05,1.0,1
3,50_75_4,2314.0,2314.0,0.0,0.45,28.0,1
4,50_75_5,22157.0,22157.0,0.0,0.05,1.0,1
5,50_75_6,10411.0,10411.0,0.0,0.36,1.0,1
6,50_75_7,21583.0,21583.0,0.0,0.07,1.0,1
7,50_75_8,3820.0,3820.0,0.0,0.08,1.0,1
8,50_75_9,22051.0,22051.0,0.0,3.80,358.0,1
9,50_75_10,10461.0,10461.0,0.0,0.73,53.0,1


In [ ]:
print(
    tab[['instance','bs','bb','gap','time','nodes','opt']].
    to_latex(index=False,float_format="%.2f")
)